In [4]:
#function for taking in a csv,cleaning, pre-processing and returning a dataframe

import pandas as pd
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from string import ascii_letters
import numpy as np
from math import sqrt
from sklearn.utils import shuffle
import sys
import time

#from sklearn.preprocessing import MinMaxScaler
#to takecare of missing values : drop cols with more than 30% NA then drop rows with NA
def preprocess(filepath):
    df = filepath
    shape = df.shape
    print("\nfile dimensions: ", shape)
    lim = 0.3*shape[0]
    features = df.columns.values
    features = features.tolist()
    print("\nColumns in dataframe: ",features)
    pick = []
    for x in features:
        print("Number of NA in " , x ,": ", df[x].isnull().sum())
        xna = df[x].isnull().sum()
        if xna < lim:
            pick.append(x)
        else:
            pass
    print("\nDROP COLUMNS WITH MORE THAN 30 PERCENT NA")
    print(len(features) - len(pick), " columns were dropped")
    print("\n",pick)

    df2 = df[pick]  #subset theh dataframe with the selected columns
    print("\nThe new dimension: ",df2.shape)

    print("\nDROP ROWS WITH NA")
    df3 = df2.dropna(axis=0, how='any') #drop rows with NA's

    print("\nClean dataframe dimension ", df3.shape[0], " rows and", df3.shape[1], " columns")

    return df3

# to drop the response col
def label(df):
    featuresLi = df.columns.values
    featuresLi = featuresLi.tolist()
    print("\nColumns in dataframe: ",featuresLi)
    label = input("What is your response variable?: ")
    response = df[label]
    featuresLi.remove(label)
    features = df[featuresLi]
    return (response, features)

def splitter_con(df):
    features = df.columns.values
    features = features.tolist()
    df_con = pd.DataFrame()
    for feature in features:
        if df[feature].dtypes == 'float64':
            df_con[feature] = df[feature]
        else:
            pass

    return df_con

def splitter_cat(df):
    features = df.columns.values
    features = features.tolist()
    df_cat = pd.DataFrame()
    for feature in features:
        if df[feature].dtypes != 'float64':
            df_cat[feature] = df[feature]
        else:
            pass

    return df_cat


def normalize(df):

    features = df.columns.values
    features = features.tolist()
    df_scaled = pd.DataFrame()
    for feature in features:
        ls = []
        for row in df[feature]:

            xs = (row - min(df[feature])) / (max(df[feature]) - min(df[feature]))
            xs = round(xs, 5)
            ls.append(xs)
        df_scaled[feature] = ls


    return df_scaled

#-----------------------------------------------------------------------------
#This function removes the columns with only unique values. e.g index numbers
def removeUcol(df):
    features = df.columns.values
    features = features.tolist()
    for x in features:
        if len(set(df[x])) == len(df[x]):
            features.remove(x)
        else:
            pass
    df = df[features]
    return df


#----------------------------------------------------------------------------
def rowsums_train(df):
    val = []
    for row in range(0,len(df)):
        v = sum(df.iloc[row])
        v = round(v,2)
        val.append(v)
    df['RowSums'] = val
    return df

#----------------------------------------------------------------------------
def rowsums_test(df):
    val = []
    for row in range(0,len(df)):
        v = sum(df.iloc[row])
        v = round(v,2)
        val.append(v)
    df['RStest'] = val
    return df


#---------------------------------------------------------------------------
#This function relabels categorical variables with a random selection of alphabets
def relabel(df):
    features = df.columns.values
    features = features.tolist()
    target = input("What feature do you want to relabel?: ")
    let = list(ascii_letters)
    for feature in features:
        if target == feature:
            print("Matched feature: ", feature)
            s = set(df[feature])
            s = list(s)
            print("This is s: ", s)
            a = len(s)
            print("This is the length: ", a)
            rlet = list(np.random.choice(let, a, replace = False))
            print("These are the random letters: ", rlet)

            c = 0
            while c < a:
                for row in df[feature]:
                    if row == s[c]:
                        df[feature] = df[feature].replace(row,rlet[c])
                    else:
                        pass

                c = c + 1


        else:
            pass

    return df
#---------------------------------------------------------------------------
#create a new column with the sets from the rows in the categorical df
def setCat(df):
    ls = []
    for row in range(0,len(df)):
        v = set(df.iloc[row])
        ls.append(v)
    df['sets'] = ls
    return df

def traintestsplit(df):
    size = len(df)
    size_train = int(0.8 * size)
    size_test = size - size_train
    df = shuffle(df)
    df_train = df[0:size_train]
    df_test = df[size_train:size]
    df_train = df_train.reset_index()
    df_test = df_test.reset_index()
    df_train = df_train.drop("index", axis = 1)
    df_test = df_test.drop("index", axis = 1)

    return (df_train, df_test)

def accuracy(df):
    correct = []
    incorrect = []
    v = df[["label","predict"]]
    for row in range(0,len(v)):
        if v.iloc[row]["predict"] != v.iloc[row]["label"]:
            incorrect.append(0)
        else:
            correct.append(1)

    a = round((len(correct)/len(v))*100,2)
    print("Accuracy score: ", a)

    
    
#df will be the merged dataframe with the continuous, set jaccard and label
def Gower(df_train, df_test):
    k = int(input("What is your k?"))
    features = df_test.columns.values
    features = features.tolist()
    features.remove("label")
    bte = len(df_test)
    btr = len(df_train)

    proc = df_train

    predict = []


    a = 0
    while a < bte:

        b = 0
        gower = []
        while b < btr:
            Mdist = []
            Jdist = []


            for feature in features:

                row_test = df_test.loc[a]
                row_train = df_train.loc[b]


                if type(row_test[feature]) != set:

                    d = (float(row_test[feature]) - float(row_train[feature]))**2

                    Mdist.append(d)


                else:

                    d = round(1 - len(row_test[feature].intersection(row_train[feature]))/len(row_test[feature].union(row_train[feature])), 4)
                    Jdist.append(d)


            g1 = round(sqrt(sum(Mdist)),5)
            g2 = Jdist[0]
            g3 = g1 + g2
            gower.append(g3)
            b = b + 1


        proc["gower"] = gower
        proc = proc.sort_values(by = "gower")
        Kproc = proc[:k]
        lab = pd.DataFrame(Kproc["label"].value_counts())
        lab = lab.sort_values(by = "label", ascending=False)
        lab = lab.reset_index()
        u = lab["index"][0]
        predict.append(u)


        a = a + 1

        print(a, "out of ", bte,": "," Actual: ", row_test["label"], "predicted: ", u)

    df_test["predict"] = predict

    return df_test







In [5]:


def probdict(df):
    col = input("Enter Response Variable: ")
    ls = list(set(df[col]))
    d = {}
    L = []
    features = df.columns.values
    features = features.tolist()
    features.remove(col)
    for i in ls:
        f = {}
        d[i] = []
        a = df.loc[(df[col]==i)]
        size = len(a)
        s = size/(len(df))
        L.append(s)
        for feature in features:
            f[feature] = []
            data = pd.DataFrame(a[feature].value_counts())
            data = data.reset_index()
            ls1 = list(set(a[feature]))
            lsd = {}
            for j in ls1:

                for row in range(0,len(data)):
                    l = []
                    if j == data.loc[row]["index"]:
                        t = data.loc[row][feature]
                        p =(t / size)
                        r = round(p,5)
                        l.append(r)
                        lsd.update({j:r})
                    else:
                        pass
                    
            f.update({feature:lsd})     
        d[i] = f    
    return (d, col, L)

def multiply(ls):  
    total = 1
    for x in ls:
        total *= x  
    return total 


def predict(dct, test, col, L): 
    features = test.columns.values
    features = features.tolist()
    features.remove(col)
    pred = []
    for row in range(0,len(test)):
        ter = test.loc[row]
        zero = []
        one = []
        for feature in features:
            l1 = ter[feature]
            try:
                x = dct[0][feature][l1]
            except:
                x = dct[0][feature][int(l1)]
                
            zero.append(x)
            
            try:
                y = dct[1][feature][l1]
            except :
                pass
            one.append(y)
        newzero = (multiply(zero))*L[0]
        newone = (multiply(one))*L[1]
        if newzero > newone:
            pred.append(0)
        else:
            pred.append(1)
        
        

    test["predict"] = pred
    return test

def naiveaccuracy(test):
    df = test[["survived","predict"]]
    acc = []
    for row in range(0,len(df)):
        drow = df.loc[row]
        if drow["predict"] == drow["survived"]:
            acc.append("accurate")
    aa = (len(acc)/len(df))* 100
    print(len(acc))
    print(len(df))
    return aa



